### Goal
The goal of this notebook is to prepare our data for the project. There are two sources of data that we are considering. One is the tabular data from the tracking data. We reformat this to be in such a way that one row represents a play. We make a huge assumption that we are only considering the offense here. The second form of data will be trying to use computer vision from the same data set. Hopefully this is able to capture the defense part.

### Importing Packages 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
import warnings
warnings.filterwarnings("ignore")

### Constructing The Tabular Data

In [7]:
# The play data 
play_data = pd.read_csv("../Data/plays.csv")

# The player data
player_data = pd.read_csv("../Data/players.csv")

# player scores 
player_scores = pd.read_csv("../Data/robust_player_scores.csv")

training_data_total = pd.read_csv("../Data/training_data_total.csv")

In [6]:
# now we need a function to do this entire process

def clean_week_data(week_data = None, play_data = None, player_data = None, player_scores = None):
    # first filter out the frames that to just before and during the snap
    pre_snap_data = week_data.loc[(week_data['displayName'] != 'football') & (week_data['frameType'] != 'AFTER_SNAP'),:]

    # we then drop the columns that we definately do not need
    pre_snap_data.drop(columns = ['displayName','frameType', 'time', 'jerseyNumber', 'event'], inplace = True)

    # we calculate the distance moved from the previous frame
    pre_snap_data["distance"] =  np.sqrt(
        pre_snap_data.groupby(['gameId', 'playId', 'nflId'])['x'].diff().abs()**2 + 
        pre_snap_data.groupby(['gameId', 'playId', 'nflId'])['y'].diff().abs()**2
        )
    
    # fill in the na with a 0 for the first rolling 
    pre_snap_data['distance'] = pre_snap_data['distance'].fillna(0)

    # we then get the total distance moved in the play
    pre_snap_data['total_distance'] = pre_snap_data.groupby(['gameId', 'playId', 'nflId'])['distance'].transform('sum')

    # drop the intermediate column from the data
    pre_snap_data.drop(columns=['distance'], inplace = True)

    # adding the player scores to the data 
    pre_snap_data_with_scores = pre_snap_data.merge(player_scores, on='nflId', how='left')
    # now we add more features to each of plays from the play data
    filtered_play_data = play_data.loc[:,["gameId","playId","down","yardsToGo","possessionTeam","yardsGained"]]

    # merge this with our data 
    pre_snap_data_with_play_data = pre_snap_data_with_scores.merge(filtered_play_data,on = ['gameId','playId'], how = 'left')

    # grab only the offensive teams
    pre_snap_data_offense_only = pre_snap_data_with_play_data.loc[pre_snap_data_with_play_data['club'] == pre_snap_data_with_play_data['possessionTeam'], :]

    # drop the column 
    pre_snap_data_offense_only.drop(columns = ['possessionTeam'], inplace = True)

    # add the player position to the data
    positions = player_data.loc[:,['nflId', 'position']]
    pre_snap_data_with_positions = pre_snap_data_offense_only.merge(positions, on = 'nflId', how = 'left')

    # drop the frame ID 
    # pre_snap_data_with_positions.drop(columns = ['frameId'], inplace = True)

    # now we will reformat the data to have one obervation be one play
    # this is the trickiest part
    # we will then be ready to try this function with every week to construct our final data set

    # each play will have 11 players
    # for each player track a unique index for every play 
    pre_snap_data_with_positions['player_index'] = pre_snap_data_with_positions.groupby(['gameId', 'playId'])['nflId'].rank(method='dense').astype(int)
    # pre_snap_data_with_positions['player_index'] = pre_snap_data_with_positions.groupby(['gameId', 'playId'])['position'].rank(method='dense').astype(int)


    # track the columns that will habe the same value throughout the entire play
    common_columns = ["gameId", "playId", "club", "playDirection", "down", "yardsToGo", "yardsGained"]

    # all of the other colunms 
    different_columns = ["gameId", "playId", "player_index", "x", "y",
                         "s", "a", "dis", "o", "dir", "total_distance", "position", "player_score"]
    
    # only keep the unqiue values for the common data
    common_data = pre_snap_data_with_positions[common_columns].drop_duplicates()

    # only keep the last frameId values 
    last_frame_data = (
        pre_snap_data_with_positions.groupby(['gameId','playId','nflId']).tail(1)
    )

    # pivot the data so that each of the rows will be one play
    pivoted_data = last_frame_data[different_columns].pivot(index=['gameId', 'playId'], columns='player_index').reset_index()

    # some renaming for the new data frame
    pivoted_data.columns = ['_'.join(map(str,col)).strip('_') for col in pivoted_data.columns]

    pre_snap_data_reformated = pd.merge(common_data, pivoted_data, on = ['gameId', 'playId'])

    # now for adding our counter factuals to the data set
    #lets first drop all the ones that are missing
    # we will then combine some of the plaued
    # notice that all the values where the coverage is missing is due to the offense kneeling
    # decide to drop sense theere is not much we can really do
    non_kneeling_play_data = play_data.loc[play_data["pff_passCoverage"].isna() != True,:]

    label_mapping = {
    'Cover-0': 'Man',
    'Cover-1': 'Man',
    'Cover-1 Double': 'Man',
    'Cover-2': 'Zone',
    '2-Man': 'Man',
    'Cover-3': 'Zone',
    'Cover-3 Seam': 'Zone',
    'Cover-3 Cloud Right': 'Zone',
    'Cover-3 Cloud Left': 'Zone',
    'Cover-3 Double Cloud': 'Zone',
    'Cover 6-Left': 'Zone',
    'Cover-6 Right': 'Zone',
    'Quarters': 'spec',
    'Red Zone': 'spec',
    'Goal Line': 'spec',
    'Bracket': 'Zone',
    'Prevent': 'Zone',
    'Miscellaneous': 'spec'
    }

    non_kneeling_play_data['assigned_defense'] = non_kneeling_play_data['pff_passCoverage'].map(label_mapping)

    # only keep the columns that are necessary to keep
    coverage_assignments =  non_kneeling_play_data.loc[:,["gameId", "playId", "assigned_defense"]]

    # merge with the other data
    final_week_data = pre_snap_data_reformated.merge(coverage_assignments, on = ['gameId','playId'], how = 'left')

    print(f"The total final data rows: {final_week_data.shape}")


    # drop the rows with no coverage assignments
    return final_week_data.dropna()




In [7]:
# we run all the data through the function and process it as one data frame 
total_training_data = []
for i in range(9):
    print(f'Running Week {i+1}')
    week = clean_week_data(week_data = pd.read_csv(f"../Data/tracking_week_{i+1}.csv"), play_data = play_data, 
                           player_data = player_data, player_scores = player_scores)
    # append to the list
    total_training_data.append(week)

Running Week 1
The total final data rows: (1952, 118)
Running Week 2
The total final data rows: (1809, 118)
Running Week 3
The total final data rows: (1959, 118)
Running Week 4
The total final data rows: (1830, 118)
Running Week 5
The total final data rows: (1906, 118)
Running Week 6
The total final data rows: (1697, 118)
Running Week 7
The total final data rows: (1665, 118)
Running Week 8
The total final data rows: (1771, 118)
Running Week 9
The total final data rows: (1535, 118)


In [8]:
# now we have to concatenate all of the training data into one data set
training_data = pd.concat(total_training_data, ignore_index=True)

In [ ]:
# export the data
# training_data.to_csv('../Data/training_data_total.csv', index=False)
# test_data.to_csv('../Data/test_data.csv', index=False)

### Creating Different Data
We will now try only keep the top 8-10 teams seince we ran into poor policy tree from using too many categorical features. We will choose this from the rankings from the previous year that the data was collected at. We could not use DAL or SF since these teams did not play in week 9. 

Top Ten Teams
1. KC
2. GB
3. TB
4. BUF
5. LA
6. NE
7. TEN
8.  IND
9.  ARI
10. CIN

In [9]:
# create a list of the top ten team names
top_ten = ['KC','GB','TB','BUF','LA','CIN','NE','TEN','ARI','IND']

# only select a row if they are in the list
top_ten_training = training_data.loc[training_data['club'].isin(top_ten),:]

In [10]:
top_ten_training.to_csv('../Data/training_data_total.csv', index=False)